In [ ]:
# Divide data into training and testing sets core    0 vs 1,2   and 1 vs 2

# Importar librerías necesarias
import pandas as pd
from sklearn.model_selection import train_test_split

# Cargar el dataset
input_file_path = "cleaned_and_scaled_dataset.csv"  # Cambiar si es necesario
df = pd.read_csv(input_file_path)

# Primera transformación: 0 vs no 0
df['y_binary_0_vs_non0'] = df['diabetes_012'].apply(lambda x: 0 if x == 0 else 1)

# Dividir en train y test para 0 vs no 0
X = df.drop(columns=['diabetes_012', 'y_binary_0_vs_non0'])  # Eliminar columnas objetivo
y_binary_0_vs_non0 = df['y_binary_0_vs_non0']

X_train_0_vs_non0, X_test_0_vs_non0, y_train_0_vs_non0, y_test_0_vs_non0 = train_test_split(
    X, y_binary_0_vs_non0, test_size=0.2, random_state=42, stratify=y_binary_0_vs_non0
)

# Contar valores en y para 0 vs no 0
print("División 0 vs no 0:")
print(f"Entrenamiento - Cantidad de 0: {sum(y_train_0_vs_non0 == 0)}, Cantidad de no 0: {sum(y_train_0_vs_non0 == 1)}")
print(f"Prueba - Cantidad de 0: {sum(y_test_0_vs_non0 == 0)}, Cantidad de no 0: {sum(y_test_0_vs_non0 == 1)}")

# Segunda transformación: 1 vs 2 (datos donde diabetes_012 es 1 o 2)
df_filtered_1_vs_2 = df[df['diabetes_012'].isin([1, 2])].copy()
df_filtered_1_vs_2['y_binary_1_vs_2'] = df_filtered_1_vs_2['diabetes_012'].apply(lambda x: 0 if x == 1 else 1)

# Dividir en train y test para 1 vs 2
X_filtered_1_vs_2 = df_filtered_1_vs_2.drop(columns=['diabetes_012', 'y_binary_1_vs_2'])
y_binary_1_vs_2 = df_filtered_1_vs_2['y_binary_1_vs_2']

X_train_1_vs_2, X_test_1_vs_2, y_train_1_vs_2, y_test_1_vs_2 = train_test_split(
    X_filtered_1_vs_2, y_binary_1_vs_2, test_size=0.2, random_state=42, stratify=y_binary_1_vs_2
)

# Contar valores en y para 1 vs 2
print("\nDivisión 1 vs 2:")
print(f"Entrenamiento - Cantidad de 1: {sum(y_train_1_vs_2 == 0)}, Cantidad de 2: {sum(y_train_1_vs_2 == 1)}")
print(f"Prueba - Cantidad de 1: {sum(y_test_1_vs_2 == 0)}, Cantidad de 2: {sum(y_test_1_vs_2 == 1)}")


División 0 vs no 0:
Entrenamiento - Cantidad de 0: 149271, Cantidad de no 0: 30043
Prueba - Cantidad de 0: 37318, Cantidad de no 0: 7511

División 1 vs 2:
Entrenamiento - Cantidad de 1: 3562, Cantidad de 2: 26481
Prueba - Cantidad de 1: 891, Cantidad de 2: 6620


In [ ]:
# train  XGBClassifier , RandomForestClassifier,  LogisticRegression  for 0 vs 1,2 

In [ ]:
#XGBClassifier for   0 vs 1,2    

# Importar librerías necesarias
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Calcular scale_pos_weight
n_zeros = sum(y_train_0_vs_non0 == 0)
n_non_zeros = sum(y_train_0_vs_non0 == 1)
scale_pos_weight = n_zeros / n_non_zeros

print(f"Proporción scale_pos_weight: {scale_pos_weight:.2f}")

# Instanciar el modelo XGBoost con scale_pos_weight
model = XGBClassifier(
    scale_pos_weight=scale_pos_weight,  # Manejo del desbalance
    use_label_encoder=False,           # Desactivar codificación de etiquetas obsoleta
    eval_metric='logloss',             # Métrica para clasificación binaria
    random_state=42,
    n_estimators=100,                  # Número de árboles
    max_depth=10,                       # Profundidad máxima de los árboles
    learning_rate=0.1                  # Tasa de aprendizaje
)

# Entrenar el modelo
model.fit(X_train_0_vs_non0, y_train_0_vs_non0)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test_0_vs_non0)

# Evaluar el modelo
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test_0_vs_non0, y_pred))

print("\nReporte de Clasificación:")
print(classification_report(y_test_0_vs_non0, y_pred))

print("\nPrecisión General del Modelo:")
print(f"{accuracy_score(y_test_0_vs_non0, y_pred):.2f}")
 

Proporción scale_pos_weight: 4.97


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [18:00:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Matriz de Confusión:
[[27040 10278]
 [ 2072  5439]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.93      0.72      0.81     37318
           1       0.35      0.72      0.47      7511

    accuracy                           0.72     44829
   macro avg       0.64      0.72      0.64     44829
weighted avg       0.83      0.72      0.76     44829


Precisión General del Modelo:
0.72


In [ ]:
#RandomForestClassifier  for 0 vs 1,2

# Importar librerías necesarias
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Instanciar el modelo Random Forest con class_weight='balanced_subsample'
model = RandomForestClassifier(
    class_weight='balanced_subsample',  # Ajuste automático de pesos por árbol
    n_estimators=100,                  # Número de árboles en el bosque
    max_depth=6,                    # Sin límite de profundidad (puedes ajustar según el dataset)
    random_state=42,                   # Semilla para reproducibilidad
    n_jobs=-1                          # Utiliza todos los núcleos disponibles para entrenar
)

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train_0_vs_non0, y_train_0_vs_non0)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test_0_vs_non0)

# Evaluar el modelo
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test_0_vs_non0, y_pred))

print("\nReporte de Clasificación:")
print(classification_report(y_test_0_vs_non0, y_pred))

print("\nPrecisión General del Modelo:")
print(f"{accuracy_score(y_test_0_vs_non0, y_pred):.2f}")



Matriz de Confusión:
[[25806 11512]
 [ 1769  5742]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.94      0.69      0.80     37318
           1       0.33      0.76      0.46      7511

    accuracy                           0.70     44829
   macro avg       0.63      0.73      0.63     44829
weighted avg       0.83      0.70      0.74     44829


Precisión General del Modelo:
0.70


In [ ]:
# LogisticRegression  for 0 vs 1,2

# Importar librerías necesarias
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Instanciar el modelo Logistic Regression con class_weight='balanced'
model = LogisticRegression(
    class_weight='balanced',  # Ajuste automático de pesos según el desbalance
    solver='lbfgs',           # Algoritmo de optimización
    max_iter=100,             # Número máximo de iteraciones
    random_state=42
)

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train_0_vs_non0, y_train_0_vs_non0)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test_0_vs_non0)

# Evaluar el modelo
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test_0_vs_non0, y_pred))

print("\nReporte de Clasificación:")
print(classification_report(y_test_0_vs_non0, y_pred))

print("\nPrecisión General del Modelo:")
print(f"{accuracy_score(y_test_0_vs_non0, y_pred):.2f}")


Matriz de Confusión:
[[26321 10997]
 [ 1817  5694]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.94      0.71      0.80     37318
           1       0.34      0.76      0.47      7511

    accuracy                           0.71     44829
   macro avg       0.64      0.73      0.64     44829
weighted avg       0.84      0.71      0.75     44829


Precisión General del Modelo:
0.71


In [ ]:


# 1 vs 2 
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Calcular scale_pos_weight
n_class_1 = sum(y_train_1_vs_2 == 0)  # Clase 1 (ahora codificada como 0)
n_class_2 = sum(y_train_1_vs_2 == 1)  # Clase 2 (ahora codificada como 1)
scale_pos_weight = n_class_2 / n_class_1

# Instanciar el modelo (sin use_label_encoder)
model = XGBClassifier(
    scale_pos_weight=15 ,
    eval_metric='aucpr',  # Métrica para clasificación binaria
    random_state=42,
    n_estimators=300,       # Número de árboles
    max_depth=15,            # Profundidad máxima de los árboles
    learning_rate=0.005      # Tasa de aprendizaje
)

# Entrenar el modelo
model.fit(X_train_1_vs_2, y_train_1_vs_2)

# Realizar predicciones
y_pred = model.predict(X_test_1_vs_2)

# Evaluar el modelo
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test_1_vs_2, y_pred))

print("\nReporte de Clasificación:")
print(classification_report(y_test_1_vs_2, y_pred))

print("\nPrecisión General del Modelo:")
print(f"{accuracy_score(y_test_1_vs_2, y_pred):.2f}")



Matriz de Confusión:
[[   1  890]
 [   4 6616]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.20      0.00      0.00       891
           1       0.88      1.00      0.94      6620

    accuracy                           0.88      7511
   macro avg       0.54      0.50      0.47      7511
weighted avg       0.80      0.88      0.83      7511


Precisión General del Modelo:
0.88


In [ ]:
#LGBMClassifier  1 vs 2 

from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Instanciar el modelo LightGBM
model = LGBMClassifier(
    is_unbalance=True,    # Manejo automático del desbalance
    random_state=42,      # Semilla para reproducibilidad
    n_estimators=100,     # Número de árboles
    max_depth=6,         # Profundidad máxima de los árboles
    learning_rate=0.01    # Tasa de aprendizaje
)

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train_1_vs_2, y_train_1_vs_2)

# Realizar predicciones
y_pred = model.predict(X_test_1_vs_2)

# Evaluar el modelo
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test_1_vs_2, y_pred))

print("\nReporte de Clasificación:")
print(classification_report(y_test_1_vs_2, y_pred))

print("\nPrecisión General del Modelo:")
print(f"{accuracy_score(y_test_1_vs_2, y_pred):.2f}")




[LightGBM] [Info] Number of positive: 26481, number of negative: 3562
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001288 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 154
[LightGBM] [Info] Number of data points in the train set: 30043, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.881437 -> initscore=2.006105
[LightGBM] [Info] Start training from score 2.006105

Matriz de Confusión:
[[  44  847]
 [  81 6539]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.35      0.05      0.09       891
           1       0.89      0.99      0.93      6620

    accuracy                           0.88      7511
   macro avg       0.62      0.52      0.51      7511
weighted avg       0.82      0.88      0.83      7511


Precisión General del Modelo:
0.88


In [ ]:
# LogisticRegression 1 vs 2 

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Instanciar el modelo Logistic Regression
model = LogisticRegression(
    class_weight='balanced',  # Ajuste automático de pesos según el desbalance
    solver='lbfgs',           # Algoritmo de optimización
    max_iter=200,             # Número máximo de iteraciones
    random_state=42           # Semilla para reproducibilidad
)

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train_1_vs_2, y_train_1_vs_2)

# Realizar predicciones
y_pred = model.predict(X_test_1_vs_2)

# Evaluar el modelo
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test_1_vs_2, y_pred))

print("\nReporte de Clasificación:")
print(classification_report(y_test_1_vs_2, y_pred))

print("\nPrecisión General del Modelo:")
print(f"{accuracy_score(y_test_1_vs_2, y_pred):.2f}")



Matriz de Confusión:
[[ 524  367]
 [2653 3967]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.16      0.59      0.26       891
           1       0.92      0.60      0.72      6620

    accuracy                           0.60      7511
   macro avg       0.54      0.59      0.49      7511
weighted avg       0.83      0.60      0.67      7511


Precisión General del Modelo:
0.60


In [ ]:
# xgboost  1 vs 2  with SMOTE
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd

# Verificar la distribución antes de SMOTE
print("Distribución antes de SMOTE (clases 1 y 2):")
print(y_train_1_vs_2.value_counts())

# Aplicar SMOTE para balancear las clases
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_1_vs_2, y_train_1_vs_2)

# Verificar la distribución después de SMOTE
print("\nDistribución después de SMOTE (clases 1 y 2):")
print(pd.Series(y_train_resampled).value_counts())

# Instanciar el modelo XGBoost
model = XGBClassifier(
    scale_pos_weight=1,        # No es necesario con SMOTE (clases ya balanceadas)
    eval_metric='aucpr',       # Métrica para datasets desbalanceados
    random_state=42,
    n_estimators=150,          # Número de árboles
    max_depth=6,              # Profundidad máxima de los árboles
    learning_rate=0.00005       # Tasa de aprendizaje
)

# Entrenar el modelo con los datos balanceados
model.fit(X_train_resampled, y_train_resampled)

# Realizar predicciones
y_pred = model.predict(X_test_1_vs_2)

# Evaluar el modelo
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test_1_vs_2, y_pred))

print("\nReporte de Clasificación:")
print(classification_report(y_test_1_vs_2, y_pred))
print("\nPrecisión General del Modelo:")
print(f"{accuracy_score(y_test_1_vs_2, y_pred):.2f}")


Distribución antes de SMOTE (clases 1 y 2):
y_binary_1_vs_2
1    26481
0     3562
Name: count, dtype: int64

Distribución después de SMOTE (clases 1 y 2):
y_binary_1_vs_2
1    26481
0    26481
Name: count, dtype: int64

Matriz de Confusión:
[[ 404  487]
 [2286 4334]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.15      0.45      0.23       891
           1       0.90      0.65      0.76      6620

    accuracy                           0.63      7511
   macro avg       0.52      0.55      0.49      7511
weighted avg       0.81      0.63      0.69      7511


Precisión General del Modelo:
0.63


In [65]:
""" 




 models 1 vs 1,2 

Random Forest:
Macro Avg Recall: 0.73
Más consistente entre ambas clases.
Logistic Regression:
Macro Avg Recall: 0.73
Similar a Random Forest, pero ligeramente menos robusto en la precisión general.
XGBoost:
Macro Avg Recall: 0.72
Menor recall en comparación con los otros modelos.








Orden de Modelos por Recall para 1 vs 2 
Basándonos en el recall balanceado entre ambas clases, el orden sería:

Logistic Regression: is the best one 
Clase 0: Recall = 0.59
Clase 1: Recall = 0.60

XGBoost (Con SMOTE):
Clase 0: Recall = 0.45
Clase 1: Recall = 0.65

LightGBM (Sin SMOTE):
Clase 0: Recall = 0.05
Clase 1: Recall = 0.99

XGBoost (Sin SMOTE):
Clase 0: Recall = 0.00
Clase 1: Recall = 1.00"""

' \n\n\n\n\n models 1 vs 1,2 \n\nRandom Forest:\nMacro Avg Recall: 0.73\nMás consistente entre ambas clases.\nLogistic Regression:\nMacro Avg Recall: 0.73\nSimilar a Random Forest, pero ligeramente menos robusto en la precisión general.\nXGBoost:\nMacro Avg Recall: 0.72\nMenor recall en comparación con los otros modelos.\n\n\n\n\n\n\n\n\nOrden de Modelos por Recall para 1 vs 2 \nBasándonos en el recall balanceado entre ambas clases, el orden sería:\n\nLogistic Regression: is the best one \nClase 0: Recall = 0.59\nClase 1: Recall = 0.60\n\nXGBoost (Con SMOTE):\nClase 0: Recall = 0.45\nClase 1: Recall = 0.65\n\nLightGBM (Sin SMOTE):\nClase 0: Recall = 0.05\nClase 1: Recall = 0.99\n\nXGBoost (Sin SMOTE):\nClase 0: Recall = 0.00\nClase 1: Recall = 1.00'

In [ ]:
# the join of best 2 model with no smote    
# Modelo 1 (0 vs no-0): 
#Modelo 2 (1 vs 2): 

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Paso 1: Cargar y preparar el dataset
import pandas as pd
input_file_path = "cleaned_and_scaled_dataset.csv"  # Cambia este archivo si es necesario
df = pd.read_csv(input_file_path)

# División 0 vs no-0
X = df.drop(columns=['diabetes_012'])  # Características
y = df['diabetes_012']  # Etiqueta objetivo

y_binary = (y != 0).astype(int)  # Etiquetas binarias para 0 vs no-0
X_train_0_vs_non0, X_test_0_vs_non0, y_train_0_vs_non0, y_test_0_vs_non0 = train_test_split(
    X, y_binary, test_size=0.2, random_state=42, stratify=y_binary
)

# División 1 vs 2
df_filtered_1_vs_2 = df[df['diabetes_012'].isin([1, 2])].copy()
X_filtered_1_vs_2 = df_filtered_1_vs_2.drop(columns=['diabetes_012'])
y_filtered_1_vs_2 = (df_filtered_1_vs_2['diabetes_012'] - 1).astype(int)  # Etiquetas binarias para 1 vs 2

X_train_1_vs_2, X_test_1_vs_2, y_train_1_vs_2, y_test_1_vs_2 = train_test_split(
    X_filtered_1_vs_2, y_filtered_1_vs_2, test_size=0.2, random_state=42, stratify=y_filtered_1_vs_2
)

# Paso 2: Entrenar modelos
# Modelo 1: Clasificador para 0 vs no-0
model_rf = RandomForestClassifier(
    class_weight='balanced_subsample',
    n_estimators=100,
    max_depth=6,
    random_state=42,
    n_jobs=-1
)
model_rf.fit(X_train_0_vs_non0, y_train_0_vs_non0)

# Modelo 2: Clasificador para 1 vs 2
model_lr = LogisticRegression(
    class_weight='balanced',
    solver='lbfgs',
    max_iter=200,
    random_state=42
)
model_lr.fit(X_train_1_vs_2, y_train_1_vs_2)

# Paso 3: Predicciones para cada modelo
# Modelo 1: Evaluar 0 vs no-0
y_pred_0_vs_non0 = model_rf.predict(X_test_0_vs_non0)
print("Modelo 1 (0 vs no-0):")
print(confusion_matrix(y_test_0_vs_non0, y_pred_0_vs_non0))
print(classification_report(y_test_0_vs_non0, y_pred_0_vs_non0))

# Modelo 2: Evaluar 1 vs 2
y_pred_1_vs_2 = model_lr.predict(X_test_1_vs_2)
print("Modelo 2 (1 vs 2):")
print(confusion_matrix(y_test_1_vs_2, y_pred_1_vs_2))
print(classification_report(y_test_1_vs_2, y_pred_1_vs_2))

# Paso 4: Modelo combinado
def combined_model_predict(X, model_rf, model_lr):
    # Predicciones del modelo 0 vs no-0
    pred_m1 = model_rf.predict(X)
    
    # Filtrar índices para 1 vs 2
    non_zero_indices = np.where(pred_m1 == 1)[0]
    final_pred = np.copy(pred_m1)  # Inicializa con las predicciones de 0 vs no-0
    
    if len(non_zero_indices) > 0:
        X_non_zero = X.iloc[non_zero_indices]  # Filtrar características no-0
        pred_m2 = model_lr.predict(X_non_zero)  # Predicciones para 1 vs 2
        final_pred[non_zero_indices] = pred_m2 + 1  # Ajustar etiquetas a 1 y 2
    
    return final_pred

# Realizar predicciones combinadas
y_pred_combined = combined_model_predict(X_test_0_vs_non0, model_rf, model_lr)

# Paso 5: Evaluar modelo combinado
print("Modelo combinado:")
print(confusion_matrix(y_test_0_vs_non0, y_pred_combined))
print(classification_report(y_test_0_vs_non0, y_pred_combined))


Modelo 1 (0 vs no-0):
[[25806 11512]
 [ 1769  5742]]
              precision    recall  f1-score   support

           0       0.94      0.69      0.80     37318
           1       0.33      0.76      0.46      7511

    accuracy                           0.70     44829
   macro avg       0.63      0.73      0.63     44829
weighted avg       0.83      0.70      0.74     44829

Modelo 2 (1 vs 2):
[[ 524  367]
 [2650 3970]]
              precision    recall  f1-score   support

           0       0.17      0.59      0.26       891
           1       0.92      0.60      0.72      6620

    accuracy                           0.60      7511
   macro avg       0.54      0.59      0.49      7511
weighted avg       0.83      0.60      0.67      7511

Modelo combinado:
[[25806  4676  6836]
 [ 1769  1550  4192]
 [    0     0     0]]
              precision    recall  f1-score   support

           0       0.94      0.69      0.80     37318
           1       0.25      0.21      0.23      7511
  

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
